In [1]:
data = '/content/drive/MyDrive/Day_03/data/IMDB.csv'


In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import sklearn

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv(data)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df.shape

(50000, 2)

In [5]:
df = df.iloc[0:10000]

In [6]:
df.shape

(10000, 2)

In [8]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [9]:
df['sentiment'].value_counts()

sentiment
positive    5028
negative    4972
Name: count, dtype: int64

In [10]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [15]:
df.duplicated().sum()

0

In [14]:
df.drop_duplicates(inplace=True)

In [16]:
df.duplicated().sum()

0

In [17]:
import re
def remove_tags(raw_text):
    clean_text = re.sub('<.*?>', '', raw_text)
    return clean_text

In [18]:
df['review'] = df['review'].apply(remove_tags)

In [19]:
df.head(1)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive


In [20]:
df['review'] = df['review'].str.lower()

In [21]:
df.review

0       one of the other reviewers has mentioned that ...
1       a wonderful little production. the filming tec...
2       i thought this was a wonderful way to spend ti...
3       basically there's a family where a little boy ...
4       petter mattei's "love in the time of money" is...
                              ...                        
9995    fun, entertaining movie about wwii german spy ...
9996    give me a break. how can anyone say that this ...
9997    this movie is a bad movie. but after watching ...
9998    this is a movie that was probably made to ente...
9999    smashing film about film-making. shows the int...
Name: review, Length: 9983, dtype: object

In [30]:
sw_list = stopwords.words('english')
df['review']= df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:' '.join(x))



In [34]:
df['review'][0]

"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. hardcore, classic use word.it called oz nickname given oswald maximum security state penitentary. focuses mainly emerald city, experimental section prison cells glass fronts face inwards, privacy high agenda. em city home many..aryans, muslims, gangstas, latinos, christians, italians, irish more....so scuffles, death stares, dodgy dealings shady agreements never far away.i would say main appeal show due fact goes shows dare. forget pretty pictures painted mainstream audiences, forget charm, forget romance...oz mess around. first episode ever saw struck nasty surreal, say ready it, watched more, developed taste oz, got accustomed high levels graphic violence. violence, injustice (crooked guards who'll sold nickel, inmates who'll kill order g

In [49]:
x = df.iloc[:,0:1]
y= df['sentiment']

In [50]:
x.review

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [51]:
y

0       positive
1       positive
2       positive
3       negative
4       positive
          ...   
9995    positive
9996    negative
9997    negative
9998    negative
9999    positive
Name: sentiment, Length: 9983, dtype: object

In [53]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [54]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)


In [55]:
x_train.shape

(7986, 1)

In [56]:
#applying bow

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()


In [57]:
x_train_bow =cv.fit_transform(x_train['review']).toarray()
x_test_bow = cv.transform(x_test['review']).toarray()

In [58]:
x_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [59]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train_bow,y_train)

GaussianNB()

In [61]:
y_pred = gnb.predict(x_test_bow)
from sklearn.metrics import accuracy_score,confusion_matrix
print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

[[717 235]
 [499 546]]


0.6324486730095142

In [63]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(x_train_bow,y_train)
y_pred = rfc.predict(x_test_bow)
from sklearn.metrics import accuracy_score,confusion_matrix
print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

[[801 151]
 [171 874]]


0.8387581372058087

In [64]:
cv = CountVectorizer(max_features=3000)
x_train_bow =cv.fit_transform(x_train['review']).toarray()
x_test_bow = cv.transform(x_test['review']).toarray()
rfc = RandomForestClassifier()
rfc.fit(x_train_bow,y_train)
y_pred = rfc.predict(x_test_bow)

accuracy_score(y_test,y_pred)

[[816 136]
 [185 860]]


0.8392588883324987

In [65]:
cv = CountVectorizer(max_features=5000,ngram_range=(1,2))
x_train_bow =cv.fit_transform(x_train['review']).toarray()
x_test_bow = cv.transform(x_test['review']).toarray()
rfc = RandomForestClassifier()
rfc.fit(x_train_bow,y_train)
y_pred = rfc.predict(x_test_bow)

print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

[[809 143]
 [164 881]]


0.8462694041061593

In [66]:
#using tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()



In [67]:
x_train_tfidf = tfidf.fit_transform(x_train['review']).toarray()
x_test_tfidf = tfidf.transform(x_test['review']).toarray()

In [68]:
rfc = RandomForestClassifier()
rfc.fit(x_train_tfidf,y_train)
y_pred = rfc.predict(x_test_tfidf)

print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

[[804 148]
 [163 882]]


0.8442663995993991

In [ ]:
#using word to vec